# Semantic Kernel Azure AI Agent Tool Use Example

This notebook shows how to build a Semantic Kernel-based tool that integrates with Azure AI Search for Retrieval-Augmented Generation (RAG) using the Azure AI Agent service. The agent retrieves travel documents from an Azure AI Search index, augments user queries with semantic search results, and streams detailed travel recommendations.

## Initializing the Environment

### Importing Packages
The following code imports the necessary packages:

In [ ]:
import json
import os

from dotenv import load_dotenv

from azure.identity import AzureCliCredential as AzureCliCredentialSync
from azure.identity.aio import AzureCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchFieldDataType,
    SearchIndex,
    SearchableField,
    SimpleField,
)

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, ChatHistoryAgentThread
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### Defining the Prompt Plugin

The `SearchPlugin` is a native Semantic Kernel plugin that augments user prompts with retrieval context from Azure AI Search.

In [3]:
class SearchPlugin:
    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context or function results.",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "First review the retrieved context, if this does not answer the query, try calling an available plugin functions that might give you an answer. If no context is available, say so."
        )

    @kernel_function(
        name="retrieve_documents",
        description="Retrieve documents from the Azure Search service.",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "No results found"

## Vector Database Initialization

Azure AI Search is initialized with persistent storage and enhanced sample documents.

In [4]:
load_dotenv()

search_service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
index_name = "travel-documents"

if not search_service_endpoint:
    raise ValueError("Azure Search endpoint must be set in the environment.")

token_credential = AzureCliCredentialSync()

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=token_credential,
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=token_credential,
)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String),
]

index = SearchIndex(name=index_name, fields=fields)

try:
    index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists, using the existing index.")
except Exception:
    print(f"Creating new index '{index_name}'...")
    index_client.create_index(index)

documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris, with average temperatures of 82°F (28°C), 45°F (7°C), and 75°F (24°C) respectively."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."},
]

search_client.upload_documents(documents)

Index 'travel-documents' already exists, using the existing index.


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is its average temperature?",
    ]

    search_plugin = SearchPlugin(search_client=search_client)

    async with (
        AzureCliCredential() as credential,
        AzureAIAgent.create_client(credential=credential, endpoint=os.getenv("AZURE_AI_PROJECT_ENDPOINT")) as client,
    ):
        settings = AzureAIAgentSettings(
            model_deployment_name=os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4.1"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_version="2024-06-01-preview",
        )

        agent_definition = await client.agents.create_agent(
            model=settings.model_deployment_name,
            name="TravelAgentHost",
            instructions="Answer travel queries using the provided tools and context. If context is provided, do not say 'I have no context for that.'",
        )

        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
            plugins=[search_plugin],
        )

        try:
            for user_input in user_inputs:
                print(f"# User: {user_input}")
                # 4. Invoke the agent with the specified message for response
                response = await agent.get_response(messages=user_input, thread=thread)
                print(f"# {response.name}: {response}")
                thread = response.thread
        finally:
            if thread:
                await thread.delete()
            await client.agents.delete_agent(agent.id)

await main()

# User: Can you explain Contoso's travel insurance coverage?
# TravelAgentHost: Contoso's travel insurance coverage includes:

- Medical emergencies
- Trip cancellations
- Lost baggage

These areas are covered to help protect travelers during unforeseen events on their trips. If you need specific benefit details or coverage limits, let me know!
# User: What is the average temperature of the Maldives?


### Creating and Running the Azure AI Agent

An Azure AI agent is provisioned dynamically, connected to the Semantic Kernel search plugin, and invoked with sample travel questions.

You should see output similar to the following:

```text
User:
Can you explain Contoso's travel insurance coverage?

Function Calls (click to expand)

Calling function: retrieve_documents({"query": "Contoso travel insurance coverage"})

Function Result:

Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.

Document: Contoso Travel provides exclusive access to boutique hotels and private guided tours.

Document: Our premium travel services include personalized itinerary planning and 24/7 concierge support.

TravelAgent:

Contoso's travel insurance coverage includes the following:

1. **Medical Emergencies**: Coverage for unforeseen medical issues that may arise while traveling.
2. **Trip Cancellations**: Protection in case you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost during your trip.

If you need more specific details about the policy, it would be best to contact Contoso directly or refer to their official documentation.
```